In [1]:
from nipype import Node, Workflow, IdentityInterface, Function, MapNode,SelectFiles, DataSink
from nipype.interfaces import fsl
from nipype.algorithms.modelgen import SpecifyModel
import numpy as np
import os

In [2]:
def sort_inputs(copes,varcopes,num_of_runs):
    
    copes.sort(key = lambda x: x.split("/")[-1])
    cope_list =  [copes[i:i + num_of_runs] for i in range(0, len(copes), num_of_runs)] 

    varcopes.sort(key = lambda x: x.split("/")[-1])
    varcope_list =  [varcopes[i:i + num_of_runs] for i in range(0, len(varcopes), num_of_runs)] 
    
    return cope_list,varcope_list
    
def get_dofvolumes(dof_files, cope_files):
    import os
    import nibabel as nb
    import numpy as np
    img = nb.load(cope_files[0])
    if len(img.shape) > 3:
        out_data = np.zeros(img.shape)
    else:
        out_data = np.zeros(list(img.shape) + [1])
    for i in range(out_data.shape[-1]):
        dof = np.loadtxt(dof_files[i])
        out_data[:, :, :, i] = dof
    filename = os.path.join(os.getcwd(), 'dof_file.nii.gz')
    newimg = nb.Nifti1Image(out_data, None, img.header)
    newimg.to_filename(filename)
    return filename

In [3]:
data = "/home/paradeisios/Desktop/nipype_output/first_level/"
output = "/home/paradeisios/Desktop/nipype_output/second_level/"
mask_file = "/home/paradeisios/fsl/data/standard/MNI152_T1_2mm_brain_mask.nii.gz"

num_of_subjects = 1
num_of_runs = 2

subject_list = ["{:02d}".format(i) for i in range(1,num_of_subjects+1)]

In [4]:
infosource = Node(IdentityInterface(fields=["subject_list"]), name="infosource")
infosource.iterables = [('subject_list', subject_list)]

In [5]:
templates = {"copes":     "modelestimate/{subject_list}/modelestimate_run_*/cope*.nii.gz",
             "varcopes":  "modelestimate/{subject_list}/modelestimate_run_*/varcope*.nii.gz",
             "dof_files" : "modelestimate/{subject_list}/modelestimate_run_*/dof"}

selectfiles = Node(SelectFiles(templates),name="selectfiles")
selectfiles.inputs.base_directory = data

In [6]:


datasink = Node(DataSink(base_directory=output),name="datasink")
#datasink.inputs.substitutions = substitutions
datasink.output = output

In [7]:
sort_copes_varcopes = Node(Function(input_names=["copes","varcopes","num_of_runs"],
                                  output_names=["cope_list","varcope_list"],
                                  function=sort_inputs),
                                  name='sort_copes_varcopes')

sort_copes_varcopes.inputs.num_of_runs = num_of_runs


gendof = Node(Function(input_names=['dof_files', 'cope_files'],
                       output_names=['dof_volume'],
                       function=get_dofvolumes),
                       name='gendofvolume')

In [8]:
copemerge = MapNode(fsl.Merge(),iterfield=['in_files'],name="copemerge")
copemerge.inputs.dimension = "t"

varcopemerge = MapNode(fsl.Merge(),iterfield=['in_files'],name="varcopemerge")
varcopemerge.inputs.dimension = "t"

level2model = Node(interface=fsl.L2Model(), name='l2model')
level2model.inputs.num_copes = num_of_runs

fixed_eff_model = MapNode(interface=fsl.FLAMEO(),name="fixed_eff_model",iterfield=['cope_file', 'var_cope_file'])
fixed_eff_model.inputs.run_mode='fe'
fixed_eff_model.inputs.mask_file= mask_file

In [9]:
fixed_effects_analysis = Workflow(name="fixed_effects_analysis")

fixed_effects_analysis.connect(infosource,"subject_list",datasink,"container")
fixed_effects_analysis.connect(infosource,"subject_list",selectfiles,"subject_list")
fixed_effects_analysis.connect(selectfiles,"copes",sort_copes_varcopes,"copes")
fixed_effects_analysis.connect(selectfiles,"varcopes",sort_copes_varcopes,"varcopes")
fixed_effects_analysis.connect(selectfiles,"dof_files",gendof,"dof_files")

fixed_effects_analysis.connect(sort_copes_varcopes,"cope_list",copemerge,"in_files")
fixed_effects_analysis.connect(sort_copes_varcopes,"varcope_list",varcopemerge,"in_files")

fixed_effects_analysis.connect(copemerge,"merged_file",fixed_eff_model,"cope_file")

fixed_effects_analysis.connect(copemerge,"merged_file",gendof,"cope_files")
fixed_effects_analysis.connect(varcopemerge,"merged_file",fixed_eff_model,"var_cope_file")

    
fixed_effects_analysis.connect(level2model,"design_mat",fixed_eff_model,"design_file")   
fixed_effects_analysis.connect(level2model,"design_con",fixed_eff_model,"t_con_file") 
fixed_effects_analysis.connect(level2model,"design_grp",fixed_eff_model,"cov_split_file") 
fixed_effects_analysis.connect(gendof,"dof_volume",fixed_eff_model,"dof_var_cope_file") 



In [10]:
fixed_effects_analysis.write_graph(graph2use='flat', format='png', simple_form=True,
                          dotfilename= os.path.join(output,"fixed_effects_analysis_workflow.dot"))

210807-02:01:08,358 nipype.workflow INFO:
	 Generated workflow graph: /home/paradeisios/Desktop/nipype_output/second_level/fixed_effects_analysis_workflow.png (graph2use=flat, simple_form=True).


'/home/paradeisios/Desktop/nipype_output/second_level/fixed_effects_analysis_workflow.png'

In [11]:
fixed_effects_analysis.connect(selectfiles,"copes",datasink,"copes")
fixed_effects_analysis.connect(selectfiles,"varcopes",datasink,"varcopes")
fixed_effects_analysis.connect(selectfiles,"dof_files",datasink,"dof_files")

fixed_effects_analysis.connect(copemerge,"merged_file",datasink,"merge.@merged_cope_file")
fixed_effects_analysis.connect(varcopemerge,"merged_file",datasink,"merge.@merged_varcope_file")

fixed_effects_analysis.connect(level2model,"design_mat",datasink,"model.@design_mat")
fixed_effects_analysis.connect(gendof,"dof_volume",datasink,"model.@dof_volume")

fixed_effects_analysis.connect(fixed_eff_model,"copes",datasink,"fixed_effects.@copes")
fixed_effects_analysis.connect(fixed_eff_model,"var_copes",datasink,"fixed_effects.@var_copes")
fixed_effects_analysis.connect(fixed_eff_model,"weights",datasink,"fixed_effects.@weights")
fixed_effects_analysis.connect(fixed_eff_model,"zstats",datasink,"fixed_effects.@zstats")
fixed_effects_analysis.connect(fixed_eff_model,"res4d",datasink,"fixed_effects.@res4d")
fixed_effects_analysis.connect(fixed_eff_model,"tstats",datasink,"fixed_effects.@tstats")

In [12]:
fixed_effects_analysis.run(plugin='MultiProc', plugin_args={'n_procs' : 2})

210807-02:01:08,430 nipype.workflow INFO:
	 Workflow fixed_effects_analysis settings: ['check', 'execution', 'logging', 'monitoring']
210807-02:01:08,456 nipype.workflow INFO:
	 Running in parallel.
210807-02:01:08,461 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 2 jobs ready. Free memory (GB): 6.74/6.74, Free processors: 2/2.
210807-02:01:08,536 nipype.workflow INFO:
	 [Node] Setting-up "fixed_effects_analysis.selectfiles" in "/tmp/tmpzfsloai2/fixed_effects_analysis/_subject_list_01/selectfiles".
210807-02:01:08,536 nipype.workflow INFO:
	 [Node] Setting-up "fixed_effects_analysis.l2model" in "/tmp/tmpzza3tb20/fixed_effects_analysis/l2model".
210807-02:01:08,549 nipype.workflow INFO:
	 [Node] Running "l2model" ("nipype.interfaces.fsl.model.L2Model")
210807-02:01:08,556 nipype.workflow INFO:
	 [Node] Running "selectfiles" ("nipype.interfaces.io.SelectFiles")
210807-02:01:08,559 nipype.workflow INFO:
	 [Node] Finished "fixed_effects_analysis.l2model".
210807-02:01:08,574 nip

210807-02:01:20,532 nipype.workflow INFO:
	 [Node] Setting-up "_copemerge1" in "/tmp/tmpz06j80p3/fixed_effects_analysis/_subject_list_01/copemerge/mapflow/_copemerge1".
210807-02:01:20,540 nipype.workflow INFO:
	 [Node] Running "_copemerge1" ("nipype.interfaces.fsl.utils.Merge"), a CommandLine Interface with command:
fslmerge -t cope2_merged.nii.gz /home/paradeisios/Desktop/nipype_output/first_level/modelestimate/01/modelestimate_run_1/cope2.nii.gz /home/paradeisios/Desktop/nipype_output/first_level/modelestimate/01/modelestimate_run_2/cope2.nii.gz
210807-02:01:20,542 nipype.workflow INFO:
	 [Node] Running "_copemerge2" ("nipype.interfaces.fsl.utils.Merge"), a CommandLine Interface with command:
fslmerge -t cope3_merged.nii.gz /home/paradeisios/Desktop/nipype_output/first_level/modelestimate/01/modelestimate_run_1/cope3.nii.gz /home/paradeisios/Desktop/nipype_output/first_level/modelestimate/01/modelestimate_run_2/cope3.nii.gz
210807-02:01:21,0 nipype.workflow INFO:
	 [Node] Finished "

210807-02:01:34,593 nipype.workflow INFO:
	 [Node] Running "_fixed_eff_model3" ("nipype.interfaces.fsl.model.FLAMEO"), a CommandLine Interface with command:
flameo --copefile=/tmp/tmpz06j80p3/fixed_effects_analysis/_subject_list_01/copemerge/mapflow/_copemerge3/cope4_merged.nii.gz --covsplitfile=/tmp/tmpzza3tb20/fixed_effects_analysis/l2model/design.grp --designfile=/tmp/tmpzza3tb20/fixed_effects_analysis/l2model/design.mat --dofvarcopefile=/tmp/tmp96cr4tne/fixed_effects_analysis/_subject_list_01/gendofvolume/dof_file.nii.gz --ld=stats --maskfile=/home/paradeisios/fsl/data/standard/MNI152_T1_2mm_brain_mask.nii.gz --runmode=fe --tcontrastsfile=/tmp/tmpzza3tb20/fixed_effects_analysis/l2model/design.con --varcopefile=/tmp/tmpcnjbhklp/fixed_effects_analysis/_subject_list_01/varcopemerge/mapflow/_varcopemerge3/varcope4_merged.nii.gz
210807-02:01:36,489 nipype.workflow INFO:
	 [MultiProc] Running 2 tasks, and 0 jobs ready. Free memory (GB): 6.34/6.74, Free processors: 0/2.
                  